# 교차검증과 그리드 서치 (Cross Validation & Grid Search)

전체 데이터에서 나눈 80%의 훈련세트에서 또 20% 를 떼어내어 검증세트로 만든다.

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
# 'class'열을 타깃으로 하고 나머지 열은 배열에 저장
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine[['class']].to_numpy()

In [3]:
# 훈련세트와 테스트세트를 나눈다.
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [4]:
# train_input, train_target을 다시 검증세트 val_input, val_target에 20%씩 나눔.
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
# 모델을 만들고 평가
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

# 과대적합이 되어있으므로 매개변수를 바꿔야한다.

0.9971133028626413
0.8625


검증세트를 만드느라 훈련세트가 줄었는데 검증세트를 너무 조금하면 불안정하다.수

그럴때는 교차검증을 이용

scikit.learn의 cross_validate()라는 교차검증함수 이용

In [7]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00498509, 0.00467086, 0.00485492, 0.00483298, 0.00446606]), 'score_time': array([0.00050497, 0.00047898, 0.00045514, 0.00045705, 0.00040197]), 'test_score': array([0.86826923, 0.85096154, 0.87295476, 0.85081809, 0.83638114])}


In [8]:
import numpy as np
print(np.mean(scores['test_score']))

0.8558769526911971


교차 검증을 할 때 훈련세트를 섞으려면 분할기(Splitter)를 지정해야한다.

앞서 수행한 교차 검증은 다음 코드와 동일

In [9]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.856646183460428


훈련세트를 섞은 후 10-폴드 교차 검증을 수행하려면 n_splits 사용

In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8576122721209426


### 하이퍼파라미터 튜닝
사용자가 지정해야만 하는 파라미터 = 하이퍼파라미터

사이킷런의 ** Grid Search **를 사용

하이퍼파라미터 탐색과 교차 검증을 한 번에 수행

별도 cross_validate() 함수를 호출할 필요가 없다.

In [11]:
# 탐새갈 매개변수와 탐색할 값의 리스트를 딕셔너리로 만든다.
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.001, 0.002, 0.003, 0.004, 0.005]}

In [12]:
# 그리드서치 객체를 만든다.
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [13]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.001, 0.002, 0.003, 0.004,
                                                   0.005]})

최적의 하이퍼파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야 하는데

사이킷런 Grid Search는 훈련이 끝나면 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련한다.

gs 객체의 best_estimatior_ 속성에 저장되어 있다.

In [14]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.8724263998460651


In [15]:
# 최적의 매개변수는 best_params_속성에 저장
print(gs.best_params_)

{'min_impurity_decrease': 0.001}


In [16]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 mean_test_score키에 저장
print(gs.cv_results_['mean_test_score'])

[0.86165044 0.85761105 0.85607111 0.85414803 0.85318557]


In [17]:
# 넘파이 argmax()함수를 사용하면 자동으로 가장 큰 값의 인덱스를 추출할 수 있다.
# 그 다음 그 인덱스를 사용하여 params 키에 저장된 매개변수를 출력할 수 있다.
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.001}


In [18]:
params = {'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001),
          'max_depth':range(5,20,1),
          'min_samples_split':range(2,100,10)}

In [19]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [20]:
print(gs.best_params_) # 최상의 매개변수 조합

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [21]:
print(np.max(gs.cv_results_['mean_test_score']))
# 최상의 교차 검증 점수

0.8683865773302731


### Random Search
매개변수를 샘플링할 수 있는 확률 분포 객체를 전달

scipy.stats의 uniform, randint

- 주어진 범위에서 고르게 값을 뽑는다.
- randint : 정수값
- uniform : 실수값

In [22]:
from scipy.stats import uniform, randint

rgen = randint(0,10)
rgen.rvs(10)
# 0~10 사이의 정수 샘플링

array([4, 7, 2, 3, 9, 3, 9, 7, 4, 0])

In [23]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([122, 109, 107,  95,  83,  93,  89, 107,  96,  99]))

In [24]:
ugen = uniform(0,1)
ugen.rvs(10)
# 실수 랜

array([0.35617349, 0.88368454, 0.10353357, 0.34213827, 0.35147195,
       0.60849395, 0.53539784, 0.49986254, 0.45630265, 0.1647301 ])

탐색할 매개변수의 딕셔너리를 만들기

min_samples_leaf 매개변수 : 리프노드가 되기 위한 최소 샘플의 개수

어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할 하지 않는다.

In [25]:
# 탐색할 매개변수 범위
params = {'min_impurity_decrease':uniform(0.0001, 0.001),
          'max_depth':randint(20,50),
          'min_samples_split' : randint(2,25),
          'min_samples_leaf':randint(1,25)}

샘플링 횟수는 RandomizedSearchCV 의 n_iter 매개변수에 지정

In [26]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ffebe085f98>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ffebfe8d860>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ffebe3f6be0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ffebe3f6780>},
                   random_state=42)

In [27]:
print(gs.best_params_)
# 최적의 매개변수 조합

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [28]:
print(np.max(gs.cv_results_['mean_test_score']))
# 최고의 교차 검증 점수

0.8695428296438884
